In [17]:
import logging
import pandas as pd
from pathlib import Path
import api_insee
import secret
from api_insee import ApiInsee
from api_insee.criteria import Field, Periodic

pd.set_option("display.max_rows", None, "display.max_columns", None)

logging.basicConfig(format='%(asctime)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [18]:
codeCommuneEtablissement = "382*"
etatAdministratifUniteLegale = "A"
categorieJuridiqueUniteLegale = "5*"
activitePrincipaleUniteLegale = "*"
activitePrincipaleEtablissement="16.10A"

In [80]:

    query = (
    Field('codeCommuneEtablissement', codeCommuneEtablissement) 
        & Field('etatAdministratifUniteLegale', etatAdministratifUniteLegale) 
        & Field('categorieJuridiqueUniteLegale', categorieJuridiqueUniteLegale) 
        & Field('activitePrincipaleUniteLegale', activitePrincipaleUniteLegale)
        & Periodic(Field("activitePrincipaleEtablissement", activitePrincipaleEtablissement))
     
 )

In [107]:
api = ApiInsee(
            key = secret.sirene['key'],
            secret = secret.sirene['secret']
        )
data = api.siret(q=query)

In [108]:
vars(data)

{'reference': False,
 '_url_params': {'q': 'codeCommuneEtablissement:382* AND etatAdministratifUniteLegale:A AND categorieJuridiqueUniteLegale:5* AND activitePrincipaleUniteLegale:* AND periode(activitePrincipaleEtablissement:16.10A)'},
 'token': <api_insee.utils.client_token.ClientToken at 0x7f3d6a820850>}

In [109]:
tot=0
etablissements=[]
for (page_index, page_result) in enumerate(data.pages(nombre=1000)):
    et=page_result['etablissements']
    tot=tot+len(et)
    logging.info("... +"+str(len(et))+" = "+str(tot))
    if len(et) >0 :
        etablissements.append(pd.json_normalize(et))


2020-12-08 07:10:44,208 ... +11 = 11
2020-12-08 07:10:44,885 ... +0 = 11


In [85]:
etablissements[0]['periodesEtablissement'][0][0]['activitePrincipaleEtablissement']

'16.10A'

In [87]:
etablissements

[       siren    nic           siret statutDiffusionEtablissement  \
 0  348477753  00019  34847775300019                            O   
 1  381611177  00019  38161117700019                            O   
 2  390237832  00014  39023783200014                            O   
 3  412081432  00012  41208143200012                            O   
 4  423972249  00019  42397224900019                            O   
 5  500736897  00017  50073689700017                            O   
 6  508397155  00013  50839715500013                            O   
 7  508397155  00021  50839715500021                            O   
 8  812968618  00015  81296861800015                            O   
 
   dateCreationEtablissement trancheEffectifsEtablissement  \
 0                1988-10-01                            NN   
 1                1991-04-02                            11   
 2                1993-01-01                            02   
 3                1997-04-01                            01  

In [89]:
tous = pd.concat(etablissements)

In [90]:
tous['periodesEtablissement']

0    [{'dateFin': None, 'dateDebut': '2018-03-06', ...
1    [{'dateFin': None, 'dateDebut': '2010-12-31', ...
2    [{'dateFin': None, 'dateDebut': '2008-01-01', ...
3    [{'dateFin': None, 'dateDebut': '2010-12-31', ...
4    [{'dateFin': None, 'dateDebut': '2011-09-30', ...
5    [{'dateFin': None, 'dateDebut': '2008-01-01', ...
6    [{'dateFin': None, 'dateDebut': '2018-12-01', ...
7    [{'dateFin': None, 'dateDebut': '2018-12-31', ...
8    [{'dateFin': None, 'dateDebut': '2016-02-21', ...
Name: periodesEtablissement, dtype: object

In [93]:
tous.loc[:, 'newPeriode'] = tous.periodesEtablissement.map(lambda x: x[0])

In [110]:
tous.loc[:,"activitePrincipaleEtablissement"]=tous.newPeriode.map(lambda x: x['activitePrincipaleEtablissement'])

In [111]:
print (tous.newact)

0    16.10A
1    16.10A
2    16.10A
3    16.10A
4    16.10A
5    16.10A
6    16.10A
7    16.10A
8    16.10A
Name: newact, dtype: object


In [112]:
tous.loc[:, 'newact'] = tous.periodesEtablissement.map(lambda x: x[0]['activitePrincipaleEtablissement'])